# General Matrix Fact training test/debug

In [1]:
import torch
import numpy as np
import pandas as pd
from data import SingleTaskGenerator
from gmf import GMFEngine

## Load data

In [2]:
path = './data/'

In [3]:
t1_interactions = pd.read_csv(path + 't1_train.csv')

In [4]:
t1_interactions.head()

,node1,node2,edge
0,151,2841,1
1,151,2874,1
2,151,780,1
3,151,1183,1
4,155,2346,1


In [5]:
x = np.load(path + 'human_feats.npy')
y = np.load(path + 'virus_feats.npy')

In [6]:
print(x.shape, y.shape)

(7209, 2799) (1107, 2799)


# Config

In [7]:
USE_GPU = True

if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

print('using device:', device)

using device: cuda


In [8]:
num_human, num_virus = x.shape[0], y.shape[0]
assert(x.shape[1] == y.shape[1])
latent_dim = x.shape[1]

In [9]:
config = {'alias': 'gmf',
          'num_epoch': 3,
          'batch_size': 32,
          'optimizer': 'adam',
          'adam_lr': 1e-3,
          'interactions': t1_interactions,
          'num_human': num_human,
          'num_virus': num_virus,
          'latent_dim': latent_dim,
          'l2_regularization': 0.01,
          'sparse': True,
          'sparsity': .1,
          'cuda': True,
          'device_id': 0}

# Training loop

In [10]:
singletaskgen = SingleTaskGenerator(t1_interactions, x, y, .90)
engine = GMFEngine(config)

In [11]:
for epoch in range(config['num_epoch']):
    print('Epoch {} starts !'.format(epoch))
    print('-' * 80)
#     train_loader = sample_generator.instance_a_train_loader(config['num_negative'], config['batch_size'])
    train_loader = singletaskgen.create_loader(config['batch_size'])
    engine.train_an_epoch(train_loader, epoch_id=epoch)
    hit_ratio, ndcg = engine.evaluate(evaluate_data, epoch_id=epoch)
    engine.save(config['alias'], epoch, hit_ratio, ndcg)

Epoch 0 starts !
--------------------------------------------------------------------------------


KeyError: tensor([814, 869, 866, 879, 847, 886,   6, 809, 808, 835, 889, 865, 890, 902,
        815,   1, 881, 825,   7, 843, 884, 841, 856, 824,   2, 851,   5, 826,
        860, 898, 857, 858], device='cuda:0')